In [1]:
#import json
#import pprint
from pathlib import Path
from functools import cache

import duckdb
import folium
import pandas as pd

In [2]:
HEALTHKIT_DB = "../data/healthkit-sqlite-2023-11-17-fix.db"

In [3]:
CAMINO_SQL = """
    SELECT id, duration, sourceName, creationDate, startDate, endDate, workout_statistics, device 
    FROM workouts 
    WHERE sourceName != 'AllTrails' 
    AND (
        (startDate >= '2023-10-10' AND startDate <= '2023-10-17' AND duration > 100) 
        OR 
        (startDate >= '2023-10-16' AND startDate <= '2023-10-17' AND duration < 100)
    )
    """

In [4]:
WALK_SQL = """
    SELECT * FROM workout_points 
    WHERE workout_id = 'WORKOUT_ID'
    """

In [5]:
CAMINO_STATS_SQL = """
    SELECT * FROM workouts WHERE sourceName != 'AllTrails' AND startDate >= '2023-10-16' AND startDate <= '2023-10-17' 
    AND duration < 100
    """

In [6]:
if Path(HEALTHKIT_DB).exists():
    con = duckdb.connect(HEALTHKIT_DB)
    con.install_extension("sqlite")
    con.load_extension("sqlite")

In [7]:
con.sql("PRAGMA show_tables").to_df() ;

In [8]:
stats_df = con.sql(CAMINO_STATS_SQL).to_df()
# pprint.pprint(stats_df["workout_statistics"].iloc[0]) 
# json.loads(stats_df["workout_statistics"].iloc[0])[0]

In [9]:
workouts_df = con.sql(CAMINO_SQL).to_df()

In [10]:
walk_ids = workouts_df["id"].values.tolist()[1:]   # Discard evening walk in San Sebastin

In [11]:
def update_map(m, df, colour="blue", line_width=3.5):
    points = df[["latitude", "longitude"]].values.tolist()
    folium.PolyLine(points, color=colour, weight=line_width, opacity=1).add_to(m)
    folium.Marker([df["latitude"].iloc[0], df["longitude"].iloc[0]]).add_to(m)
    return m

In [25]:
def create_walk_map(walk_ids, colour, line_width):
    m = folium.Map(location=[43.3183, -1.9812], zoom_start=12, tiles="openstreetmap")
    for i, id in enumerate(walk_ids):
        print(f"#{i+1} - Getting data for walk ID: {id}")
        walk_df = con.sql(WALK_SQL.replace("WORKOUT_ID", id)).to_df()
        # TODO: Cache / preprocess the walk data
        m = update_map(m, walk_df, colour, line_width)
    m.fit_bounds(m.get_bounds())
    return m

In [26]:
# Add controls to UI for these

colour = "red"
line_width = 4.0

In [27]:
map = create_walk_map(walk_ids, colour, line_width)

#1 - Getting data for walk ID: 38ce096be81be8d33af888ef88685470cc48c2ad
#2 - Getting data for walk ID: d99d6a938ea9e6a9deab1d83b1e0b75a185994d7
#3 - Getting data for walk ID: ac3f4763d2390eaab764dd962877ddb95596b90d
#4 - Getting data for walk ID: 79367dfb9f1b4df667d1d6012c615de15a1c65df
#5 - Getting data for walk ID: 621824acbd8b06ff3de5f6dfa8632660b3a6f508
#6 - Getting data for walk ID: 9c647717fced037f54e9b080ec80827aa7e91045
#7 - Getting data for walk ID: ba92efd6a7a1f979eb95daf707d9121d6832fc20
#8 - Getting data for walk ID: a7c6cfbaace2707bba516537063c80e32596b7ee
#9 - Getting data for walk ID: 6f1556d4510b843c28330c2905bed86711dbfcda


In [24]:
map